<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-Data" data-toc-modified-id="Read-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read Data</a></span></li><li><span><a href="#Map-Test" data-toc-modified-id="Map-Test-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Map Test</a></span></li><li><span><a href="#Streets-and-Roads-Data" data-toc-modified-id="Streets-and-Roads-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Streets and Roads Data</a></span></li><li><span><a href="#Check-which-roads-can-be-found-in-Tripoli" data-toc-modified-id="Check-which-roads-can-be-found-in-Tripoli-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Check which roads can be found in Tripoli</a></span><ul class="toc-item"><li><span><a href="#Map-streets-to-Baladiay-Level" data-toc-modified-id="Map-streets-to-Baladiay-Level-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Map streets to Baladiay Level</a></span></li><li><span><a href="#Upload-to-BQ" data-toc-modified-id="Upload-to-BQ-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Upload to BQ</a></span></li></ul></li><li><span><a href="#Upload-To-BQ" data-toc-modified-id="Upload-To-BQ-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Upload To BQ</a></span></li><li><span><a href="#Upload-GEo-to-BQ" data-toc-modified-id="Upload-GEo-to-BQ-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Upload GEo to BQ</a></span></li></ul></div>

In [288]:
import pandas as pd
import numpy as np

import geopandas as gpd
import unicodedata
import re
from datetime import datetime, timedelta, time

In [289]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import fiona
import folium

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Read Data

In [290]:
# Celaned and processed data

df_pop_lvl_1_2022_cleaned = pd.read_excel("../data/MLC_libya_admin_levels_demo_data_2022.xlsx",
                             sheet_name="Level 1 Admin Data",
                            header=0)

df_pop_lvl_2_2022_cleaned = pd.read_excel("../data/MLC_libya_admin_levels_demo_data_2022.xlsx",
                             sheet_name="Level 2 Admin Data",
                            header=0)

df_pop_lvl_3_2022_cleaned = pd.read_excel("../data/MLC_libya_admin_levels_demo_data_2022.xlsx",
                             sheet_name="Level 3 Admin Data",
                            header=0)


In [306]:
#Area data
df_area_lvl_2_2018 = pd.read_csv("../data/admin_level_1_sizes.csv")
df_area_lvl_2_2018.tail()

,Date,ValidOn,ValidTo,ADM2_EN,ADM2_AR,ADM2_PCODE,ADM2_REF,ADM2ALT1_EN,ADM2ALT2_EN,ADM2ALT1_AR,ADM2ALT2_AR,ADM1_EN,ADM1_AR,ADM1_PCODE,ADM0_EN,ADM0_AR,ADM0_PCODE,AREA_SQKM
17,"11/30/2016, 1:00 AM","5/7/2018, 2:00 AM",NaN,Wadi Ashshati,وادى الشاطئ,LY0318,NaN,NaN,NaN,NaN,NaN,South,الجنوب,LY03,Libya,ليبيا,LY,"101,633.56"
18,"11/30/2016, 1:00 AM","5/7/2018, 2:00 AM",NaN,Sebha,سبها,LY0319,NaN,NaN,NaN,NaN,NaN,South,الجنوب,LY03,Libya,ليبيا,LY,"17,232.10"
19,"11/30/2016, 1:00 AM","5/7/2018, 2:00 AM",NaN,Ubari,اوباري,LY0320,NaN,NaN,NaN,NaN,NaN,South,الجنوب,LY03,Libya,ليبيا,LY,"32,407.64"
20,"11/30/2016, 1:00 AM","5/7/2018, 2:00 AM",NaN,Murzuq,مرزق,LY0322,NaN,NaN,NaN,NaN,NaN,South,الجنوب,LY03,Libya,ليبيا,LY,"280,755.38"
21,"11/30/2016, 1:00 AM","5/7/2018, 2:00 AM",NaN,Ghat,غات,LY0321,NaN,NaN,NaN,NaN,NaN,South,الجنوب,LY03,Libya,ليبيا,LY,"50,633.90"


In [301]:
#append area data to lvl 1 data

area_lvl1_mapping_dict = dict(zip(df_area_lvl_2_2018["ADM2_PCODE"], df_area_lvl_2_2018["AREA_SQKM"]))

In [303]:
df_pop_lvl_1_2022_cleaned["area_sqkm"] = df_pop_lvl_1_2022_cleaned["adm_code_lvl_1"].map(area_lvl1_mapping_dict)




In [305]:
df_pop_lvl_1_2022_cleaned.tail()

,country,geo,geo_ar,admin_lvl_1,admin_lvl_1_ar,adm_code_lvl_1,sex,age_group,population,area_sqkm
133,LIBYA,South,الجنوب,Wadi Ashshati,وادى الشاطئ,LY0318,Female,60 and older,3075,"101,633.56"
134,LIBYA,South,الجنوب,Wadi Ashshati,وادى الشاطئ,LY0318,Female,under 18,20400,"101,633.56"
135,LIBYA,South,الجنوب,Wadi Ashshati,وادى الشاطئ,LY0318,Male,18-59,26337,"101,633.56"
136,LIBYA,South,الجنوب,Wadi Ashshati,وادى الشاطئ,LY0318,Male,60 and older,2763,"101,633.56"
137,LIBYA,South,الجنوب,Wadi Ashshati,وادى الشاطئ,LY0318,Male,under 18,20806,"101,633.56"


In [307]:
# number of available muhafazat
nr_muhafaza_expected = 22
df_pop_lvl_3_2022_cleaned["admin_lvl_1"].nunique() == nr_muhafaza_expected

True

Mahallas: 
    - Point data only
    - Exception Derna - Polygon data
Baladiya:
    - Point data only
Mantika:
    -Polygons

# Map Test

In [375]:
import fiona

def list_layers(dataset_path):

    # Open the file using Fiona to list all layers
    with fiona.open(dataset_path, 'r') as dataset:
        for layername in dataset.keys():
            print("Layer:", layername)

In [308]:
def map_data(data_path, layer_name):

    gdf = gpd.read_file(data_path, layer=layer_name)
    
    if any(x in ["validOn", "validTo", "date"] for x in gdf.columns):
        gdf = gdf.drop(columns=[x for x in gdf.columns if x in ["validOn", "validTo", "date"]])


    # Convert GeoDataFrame to GeoJSON
    gdf_geojson = gdf.to_json()

    # Create a map object
    m = folium.Map(location=[gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()], zoom_start=6)

    # Add the GeoJSON overlay
    folium.GeoJson(gdf_geojson, name='geojson').add_to(m)

    # Add layer control
    folium.LayerControl().add_to(m)

    # Display map
    m.save('map.html')
    m  # Display in an interactive cell in a Jupyter notebook or open map.html in a browser
    
    return m, gdf


In [309]:
path_admin_lvl_1_poly = "../data/GeoData/admin_lvl_1_polygons.gdb" #boundries - district level
path_admin_lvl_2_point = '../data/GeoData/lby_pplp_unosat_lbsc_20181030.gdb' #point coordintaes - municipality level
path_admin_lvl_4_point = '../data/GeoData/neighborhoods_incomplete.gpkg'# neighborhood level boundries - Derna + Albeyda only



In [476]:
gpd.read_file(path_admin_lvl_4_point)

,OBJECTID,name_neigh,mantika,baladiya,muhalla,town,source,date_data,admin2Name,admin2Na_1,admin2Pcod,admin1Name,admin1Na_1,admin1Pcod,admin0Name,admin0Na_1,admin0Pcod,Shape_Leng,geometry
0,1.0,Andalus,Al Jabal Al Akhdar,Al Bayda,,Al Bayda,REACH field officers/ACTED area coordinators,2017,Al Jabal Al Akhdar,الجبل الاخضر,LY0106,East,الشرق,LY01,Libya,ليبيا,LY,4.932729,"MULTIPOLYGON (((21.75690 32.77348, 21.75045 32..."
1,2.0,Villas 300,Al Jabal Al Akhdar,Al Bayda,,Al Bayda,REACH field officers/ACTED area coordinators,2017,Al Jabal Al Akhdar,الجبل الاخضر,LY0106,East,الشرق,LY01,Libya,ليبيا,LY,4.932729,"MULTIPOLYGON (((21.75690 32.77348, 21.75724 32..."
2,3.0,Ganien,Al Jabal Al Akhdar,Al Bayda,,Al Bayda,REACH field officers/ACTED area coordinators,2017,Al Jabal Al Akhdar,الجبل الاخضر,LY0106,East,الشرق,LY01,Libya,ليبيا,LY,4.932729,"MULTIPOLYGON (((21.76729 32.77177, 21.76587 32..."
3,4.0,Shadda Alzawiya,Al Jabal Al Akhdar,Al Bayda,,Al Bayda,REACH field officers/ACTED area coordinators,2017,Al Jabal Al Akhdar,الجبل الاخضر,LY0106,East,الشرق,LY01,Libya,ليبيا,LY,4.932729,"MULTIPOLYGON (((21.76490 32.76589, 21.76473 32..."
4,5.0,New Al Bayda,Al Jabal Al Akhdar,Al Bayda,,Al Bayda,REACH field officers/ACTED area coordinators,2017,Al Jabal Al Akhdar,الجبل الاخضر,LY0106,East,الشرق,LY01,Libya,ليبيا,LY,4.932729,"MULTIPOLYGON (((21.77820 32.76543, 21.77820 32..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,0.0,Al-Dahr Al-Ahmar,Derna,Derna,Al-Maghar,Derna,FGD,2022,Derna,درنه,LY0101,East,الشرق,LY01,Libya,ليبيا,LY,0.000000,"MULTIPOLYGON (((22.62655 32.75162, 22.62762 32..."
379,0.0,Imbikh Al-Eimarat,Derna,Derna,alsalam,Derna,FGD,2022,Derna,درنه,LY0101,East,الشرق,LY01,Libya,ليبيا,LY,0.000000,"MULTIPOLYGON (((22.60331 32.77404, 22.60258 32..."
380,0.0,Imbikh Al-Quruth,Derna,Derna,alsalam,Derna,FGD,2022,Derna,درنه,LY0101,East,الشرق,LY01,Libya,ليبيا,LY,0.000000,"MULTIPOLYGON (((22.60331 32.77404, 22.60003 32..."
381,0.0,Alsayida Khadija,Derna,Derna,Alsayida Khadija,Derna,FGD,2022,Derna,درنه,LY0101,East,الشرق,LY01,Libya,ليبيا,LY,0.000000,"MULTIPOLYGON (((22.55251 32.79114, 22.55434 32..."


In [478]:
m, gdf = map_data(path_admin_lvl_4_point, "")
m

ValueError: Null layer: '1'

# Streets and Roads Data

In [390]:
streets_path = "../data/Roads_and_Streets/lby_trs_streets_osm/"
roads_path = "../data/Roads_and_Streets/lby_trs_roads_osm"

df_streets = gpd.read_file(streets_path)
df_roads = gpd.read_file(roads_path)



In [392]:
df_streets.columns

Index(['objectid', 'osm_id', 'sourceid', 'notes', 'onme', 'rtenme', 'ntlclass', 'fclass', 'numlanes', 'srftpe', 'srfcond', 'isseasonal', 'curntprac', 'gnralspeed', 'rdwidthm', 'status', 'iselevated', 'iso3', 'country', 'last_updat', 'geometry'], dtype='object')

In [393]:
df_streets.head()

,objectid,osm_id,sourceid,notes,onme,rtenme,ntlclass,fclass,numlanes,srftpe,srfcond,isseasonal,curntprac,gnralspeed,rdwidthm,status,iselevated,iso3,country,last_updat,geometry
0,589,153106506.0,None,None,None,None,residential,5,NaN,None,None,None,None,NaN,NaN,None,0,LBY,Libya,2021-01-17 12:22:19.212,"LINESTRING (12.48731 32.79525, 12.48716 32.795..."
1,827,410985545.0,None,None,None,None,residential,5,NaN,asphalt,None,None,None,NaN,NaN,None,0,LBY,Libya,2021-01-17 12:22:19.212,"LINESTRING (12.48623 32.79448, 12.48566 32.794..."
2,1461,670002712.0,None,None,None,None,residential,5,NaN,None,None,None,None,NaN,NaN,None,0,LBY,Libya,2021-01-17 12:22:19.212,"LINESTRING (12.41461 32.80692, 12.41454 32.807..."
3,1885,894531435.0,None,None,None,None,residential,5,NaN,None,None,None,None,NaN,NaN,None,0,LBY,Libya,2021-01-17 12:22:19.212,"LINESTRING (11.35424 31.99503, 11.35420 31.995..."
4,2054,893784815.0,None,None,None,None,service,5,NaN,None,None,None,None,NaN,NaN,None,0,LBY,Libya,2021-01-17 12:22:19.212,"LINESTRING (13.99643 31.71561, 13.99596 31.715..."


In [395]:
df_streets.shape[0]

219304

In [394]:
df_streets.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 219304 entries, 0 to 219303
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   objectid    219304 non-null  int64   
 1   osm_id      219304 non-null  float64 
 2   sourceid    0 non-null       object  
 3   notes       6 non-null       object  
 4   onme        0 non-null       object  
 5   rtenme      1378 non-null    object  
 6   ntlclass    219304 non-null  object  
 7   fclass      219304 non-null  int64   
 8   numlanes    162 non-null     float64 
 9   srftpe      18135 non-null   object  
 10  srfcond     744 non-null     object  
 11  isseasonal  0 non-null       object  
 12  curntprac   0 non-null       object  
 13  gnralspeed  1023 non-null    float64 
 14  rdwidthm    27 non-null      float64 
 15  status      0 non-null       object  
 16  iselevated  219304 non-null  int64   
 17  iso3        219304 non-null  object  
 18  country     2193

In [434]:
df_roads.to_csv("../data/roads_test.csv", index=False)
df_pop_lvl_1_2022_cleaned_poly.to_csv("../data/level1.csv", index=False)

# Check which roads can be found in Tripoli

In [430]:
from shapely.geometry import Polygon, LineString, Point

In [431]:
all_streets = df_streets["geometry"].set_crs("EPSG:4326") 
areas = gpd.GeoSeries(df_pop_lvl_1_2022_cleaned_poly["geometry"]).set_crs("EPSG:4326") 


In [443]:
type(gpd.GeoSeries(districts["geometry"]))

geopandas.geoseries.GeoSeries

In [447]:
roads = df_streets.copy()
districts = gpd.GeoDataFrame(df_pop_lvl_1_2022_cleaned_poly).copy()

districts["geometry"] = gpd.GeoSeries(districts["geometry"]) 
roads["geometry"] = gpd.GeoSeries(roads["geometry"]) 

# Ensure both GeoDataFrames use the same CRS
roads["geometry"].set_crs(gpd.GeoSeries(districts["geometry"]).crs, inplace=True)

# Step 3: Perform the spatial join
# This will assign each road to a district based on intersection
roads_with_districts = gpd.sjoin(roads, districts, how="left", predicate='intersects')


print("New dataset with roads and their corresponding districts has been saved.")

New dataset with roads and their corresponding districts has been saved.


In [470]:
gpd.GeoSeries(districts["geometry"]).crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [468]:
roads_with_districts["sourceid"].unique()

array([None], dtype=object)

In [469]:
roads_with_districts[['country_right',
                      'geo',
                      'geo_ar',
                      'admin_lvl_1',
                      'admin_lvl_1_ar',
                      'adm_code_lvl_1',
                      'objectid',
                      'osm_id',
                      'ntlclass',
                      'fclass',
                      'numlanes',
                      'srftpe',
                      'srfcond',
                      'gnralspeed',
                      'rdwidthm',
                      'iselevated',
                      'last_updat',
                      'geometry']].drop_duplicates(keep="first")
                     
                     

,country_right,geo,geo_ar,admin_lvl_1,admin_lvl_1_ar,adm_code_lvl_1,objectid,osm_id,ntlclass,fclass,numlanes,srftpe,srfcond,gnralspeed,rdwidthm,iselevated,last_updat,geometry
0,LIBYA,West,الغرب,Zwara,زوارة,LY0215,589,153106506.0,residential,5,NaN,None,None,NaN,NaN,0,2021-01-17 12:22:19.212,"LINESTRING (12.48731 32.79525, 12.48716 32.795..."
1,LIBYA,West,الغرب,Zwara,زوارة,LY0215,827,410985545.0,residential,5,NaN,asphalt,None,NaN,NaN,0,2021-01-17 12:22:19.212,"LINESTRING (12.48623 32.79448, 12.48566 32.794..."
2,LIBYA,West,الغرب,Zwara,زوارة,LY0215,1461,670002712.0,residential,5,NaN,None,None,NaN,NaN,0,2021-01-17 12:22:19.212,"LINESTRING (12.41461 32.80692, 12.41454 32.807..."
3,LIBYA,West,الغرب,Nalut,نالوت,LY0209,1885,894531435.0,residential,5,NaN,None,None,NaN,NaN,0,2021-01-17 12:22:19.212,"LINESTRING (11.35424 31.99503, 11.35420 31.995..."
4,LIBYA,West,الغرب,Misrata,مصراته,LY0214,2054,893784815.0,service,5,NaN,None,None,NaN,NaN,0,2021-01-17 12:22:19.212,"LINESTRING (13.99643 31.71561, 13.99596 31.715..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219299,LIBYA,East,الشرق,Derna,درنه,LY0101,17579901,661388856.0,path,7,NaN,asphalt,None,NaN,NaN,0,2021-01-15 23:29:54.478,"LINESTRING (22.66949 32.71640, 22.66626 32.716..."
219300,LIBYA,East,الشرق,Derna,درنه,LY0101,17579902,237575698.0,path,7,NaN,None,None,NaN,NaN,0,2021-01-15 23:29:54.478,"LINESTRING (22.65534 32.74966, 22.65552 32.75013)"
219301,LIBYA,East,الشرق,Derna,درنه,LY0101,17579903,237845876.0,path,7,NaN,None,None,NaN,NaN,0,2021-01-15 23:29:54.478,"LINESTRING (22.67015 32.74645, 22.67092 32.74628)"
219302,LIBYA,East,الشرق,Derna,درنه,LY0101,17579904,237845877.0,path,7,NaN,None,None,NaN,NaN,0,2021-01-15 23:29:54.478,"LINESTRING (22.67088 32.74597, 22.67070 32.745..."


In [ ]:
df = df_streets.copy()

# Convert the 'geometry' column from WKT string to LineString objects if necessary
if isinstance(df.iloc[0]['geometry'], str):
    df['geometry'] = df['geometry'].apply(wkt.loads)

# Convert DataFrame to GeoDataFrame if not already
if not isinstance(df, gpd.GeoDataFrame):
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
else:
    gdf = df

# Set the CRS if known (e.g., WGS 84 Latitude/Longitude)
gdf.set_crs("EPSG:4326", inplace=True)  # Adjust the EPSG code based on your specific dataset

# Plotting
fig, ax = plt.subplots(figsize=(10, 10))  # You can adjust the size to better fit your data
gdf.plot(ax=ax, linewidth=1, color='blue')  # Adjust color and linewidth as necessary

plt.title('Visualization of LineStrings')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)  # Optional: turn on the grid for better readability
plt.show()


## Map streets to Baladiay Level

In [366]:
from shapely.geometry import Point, LineString
import shapely

# Load the linestring data
linestrings_df = df_streets.copy()  # Adjust filename as necessary
#linestrings_df['geometry'] = linestrings_df['geometry'].apply(shapely.wkt.loads)
linestrings_gdf = gpd.GeoDataFrame(linestrings_df, geometry='geometry')

# Load the points data
points_df = df_pop_lvl_2_2022_cleaned.copy()  # Adjust filename as necessary
# Assume columns 'lat' and 'lon' and 'label'
points_df['geometry'] = points_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
points_gdf = gpd.GeoDataFrame(points_df, geometry='geometry')


In [370]:
points_gdf['buffered_geometry'] = points_gdf.geometry.buffer(0.01)  # buffer in degrees, adjust size as necessary


# If CRS of points_gdf is None, set it (usually to the CRS of linestrings_gdf if that's appropriate)
if points_gdf.crs is None:
    points_gdf.set_crs(linestrings_gdf.crs, inplace=True)


# Check CRS of both GeoDataFrames
print("CRS of linestrings_gdf:", linestrings_gdf.crs)
print("CRS of points_gdf:", points_gdf.crs)

# Perform the spatial join with updated CRS
joined_gdf = gpd.sjoin(points_gdf, linestrings_gdf, how="inner")


CRS of linestrings_gdf: EPSG:4326
CRS of points_gdf: EPSG:4326


/var/folders/vp/w0q6bwv17zg47jg_mhc4nsdh0000gp/T/ipykernel_11838/3551956362.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  points_gdf['buffered_geometry'] = points_gdf.geometry.buffer(0.01)  # buffer in degrees, adjust size as necessary


In [371]:
joined_gdf

,country_left,geo,geo_ar,admin_lvl_1,admin_lvl_1_ar,adm_code_lvl_1,admin_lvl_2,admin_lvl_2_ar,adm_code_lvl_2,sex,age_group,population,lat,lon,geometry,buffered_geometry,index_right,objectid,osm_id,sourceid,notes,onme,rtenme,ntlclass,fclass,numlanes,srftpe,srfcond,isseasonal,curntprac,gnralspeed,rdwidthm,status,iselevated,iso3,country_right,last_updat


## Upload to BQ


In [ ]:
gdf = gdf.rename(columns={"admin2Pcode":"adm_code_lvl_1"})
df_pop_lvl_1_2022_cleaned_poly = df_pop_lvl_1_2022_cleaned.merge(gdf[["adm_code_lvl_1", "Shape_Length", "Shape_Area", "geometry"]],
                                how="left",
                                on="adm_code_lvl_1")




df_pop_lvl_1_2022_cleaned_poly.to_csv("../data/output/test_adim_lvl_2.csv", index=False)

In [323]:

data = df_pop_lvl_1_2022_cleaned_poly.copy()
geo_data = gpd.GeoDataFrame(data, geometry='geometry')

geo_data['geometry'] = geo_data['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01, preserve_topology=True))


In [325]:
geo_data['geometry'] = geo_data['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01, preserve_topology=True))


In [326]:
import geopandas as gpd
from shapely.geometry import LineString
from shapely.ops import split

# Load your GeoDataFrame, assuming it's already loaded as `geo_data`
geo_data.set_crs("EPSG:4326", inplace=True)  # Ensure it's using the correct CRS

# Simplify and validate geometries
geo_data['geometry'] = geo_data['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01, preserve_topology=True) if geom.is_valid else geom.buffer(0))

# Function to handle antimeridian crossing
def handle_antimeridian(geom):
    if geom.bounds[2] > 180:
        antimeridian = LineString([(180, -90), (180, 90)])
        geom = split(geom, antimeridian)
    return geom

geo_data['geometry'] = geo_data['geometry'].apply(handle_antimeridian)

# Convert to WKT for BigQuery
geo_data['geometry'] = geo_data['geometry'].apply(lambda geom: geom.wkt)


/Users/Moe/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [327]:
from shapely import wkt

In [328]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Path to your service account key file
service_account_path = '/Users/Moe/code/moe221/gcp/taqarib-453c46fd0910.json'

# Authenticate and create a BigQuery client
credentials = service_account.Credentials.from_service_account_file(service_account_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [329]:
def detect_schema(df):
    """
    Automatically detect the schema from a DataFrame and return BigQuery fields.
    """
    type_mapping = {
        'object': 'STRING',
        'int64': 'INTEGER',
        'float64': 'FLOAT',
        'bool': 'BOOLEAN',
        'datetime64[ns]': 'DATETIME',
        'timedelta[ns]': 'STRING'  # Example handling, convert to string or duration as needed
    }
    
    schema = []
    for column, dtype in df.dtypes.iteritems():
        bq_type = type_mapping.get(str(dtype), 'STRING')  # Default to STRING if unknown
        if column == 'geometry':  # Special handling for geometry columns
            bq_type = 'GEOGRAPHY'
        schema.append(bigquery.SchemaField(column, bq_type))
    
    return schema

In [336]:
schema = detect_schema(gdf_admin_lvl_1)


In [282]:
# Define your BigQuery dataset and table
project_id = 'taqarib'
dataset_id = 'taqarib'
table_id = 'taqarib_admin_lvl_1_pop'
table_ref = client.dataset(dataset_id).table(table_id)

# Define your table schema
# Here, adjust the schema according to the types of your columns in the DataFrame
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",  # Options: WRITE_TRUNCATE, WRITE_APPEND, WRITE_EMPTY
)

# Load the DataFrame to BigQuery
job = client.load_table_from_dataframe(gdf_admin_lvl_1, table_ref, job_config=job_config)
job.result()  # Wait for the job to complete

print("Loaded {} rows into {}:{}".format(job.output_rows, dataset_id, table_id))


Loaded 138 rows into taqarib:taqarib_test


In [337]:
schema

[SchemaField('country', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('geo', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('geo_ar', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('admin_lvl_1', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('admin_lvl_1_ar', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('adm_code_lvl_1', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('sex', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('age_group', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('population', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('area_sqkm', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('Shape_Length', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('Shape_Area', 'FLOAT', 'NULLABLE', None, None, (), None),
 SchemaField('geometry', 'GEOGRAPHY', 'NULLABLE', None, None, (), None)]

# Upload To BQ

In [472]:
df_pop_lvl_2_2022_cleaned['geometry'] = df_pop_lvl_2_2022_cleaned.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
df_pop_lvl_3_2022_cleaned['geometry'] = df_pop_lvl_3_2022_cleaned.apply(lambda row: Point(row['lon'], row['lat']), axis=1)


In [473]:
df_pop_lvl_2_2022_cleaned = gpd.GeoDataFrame(df_pop_lvl_2_2022_cleaned, geometry='geometry')
df_pop_lvl_2_2022_cleaned.to_file("../data/output/df_pop_lvl_2_2022_cleaned.gpkg", driver="GPKG")

df_pop_lvl_3_2022_cleaned = gpd.GeoDataFrame(df_pop_lvl_3_2022_cleaned, geometry='geometry')
df_pop_lvl_3_2022_cleaned.to_file("../data/output/df_pop_lvl_3_2022_cleaned.gpkg", driver="GPKG")

In [339]:
df_pop_lvl_2_2022_cleaned


project_id = 'taqarib'
dataset_id = 'taqarib'
table_id = 'taqarib_admin_lvl_2_pop'


service_account_path = '/Users/Moe/code/moe221/gcp/taqarib-453c46fd0910.json'

# Authenticate and create a BigQuery client
credentials = service_account.Credentials.from_service_account_file(service_account_path)

df_pop_lvl_2_2022_cleaned.to_gbq(
    f"{dataset_id}.{table_id}",
    project_id,
    if_exists="replace",
    credentials=credentials,
)

100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2003.97it/s]


In [340]:
df_pop_lvl_3_2022_cleaned


project_id = 'taqarib'
dataset_id = 'taqarib'
table_id = 'taqarib_admin_lvl_3_pop'


service_account_path = '/Users/Moe/code/moe221/gcp/taqarib-453c46fd0910.json'

# Authenticate and create a BigQuery client
credentials = service_account.Credentials.from_service_account_file(service_account_path)

df_pop_lvl_3_2022_cleaned.to_gbq(
    f"{dataset_id}.{table_id}",
    project_id,
    if_exists="replace",
    credentials=credentials,
)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13706.88it/s]


# Upload GEo to BQ 


- Step 1: Export data as gpkg

gdf_admin_lvl_1 = gpd.GeoDataFrame(df_pop_lvl_1_2022_cleaned_poly, geometry='geometry')
gdf_admin_lvl_1.to_file("../data/output/test_adim_lvl_2.gpkg", driver="GPKG")


- Step 2:  Run this command to convert to geojsonl 

ogr2ogr -f "GeoJSONSeq" output_data.geojsonl input_data.shp

- Step 3: use code below to upload to BQ


In [331]:
gdf_admin_lvl_1 = gpd.GeoDataFrame(df_pop_lvl_1_2022_cleaned_poly, geometry='geometry')
gdf_admin_lvl_1.to_file("../data/output/test_adim_lvl_2.gpkg", driver="GPKG")


In [332]:

from google.cloud import bigquery

def load_geojsonl_to_bigquery(dataset_name, table_name, file_path):
    # Path to your service account key file
    service_account_path = '/Users/Moe/code/moe221/gcp/taqarib-453c46fd0910.json'

    # Authenticate and create a BigQuery client
    credentials = service_account.Credentials.from_service_account_file(service_account_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)

    # Configure the load job
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
        autodetect=True,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
        json_extension='GEOJSON'
    )

    # Define dataset and table ID
    dataset_ref = client.dataset(dataset_name)
    table_ref = dataset_ref.table(table_name)

    # Use the file path of the GeoJSONL
    with open(file_path, "rb") as source_file:
        job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

    # Wait for the job to complete
    job.result()  # Waits for table load to complete.
    print("Loaded {} rows into {}:{}".format(job.output_rows, dataset_name, table_name))



In [334]:
load_geojsonl_to_bigquery("taqarib", "taqarib_admin_lvl_1_pop", "/Users/Moe/code/moe221/taqarib/data/output/output_data_lvl_1.geojsonl")


Loaded 138 rows into taqarib:taqarib_admin_lvl_1_pop
